# Classificação - Satisfação de Passageiro em Linhas Aéreas

## Autores
- Felipe Bakowski Nantes de Souza  
- Vinicius Grecco Fonseca Mulato  
- Victor Soares


# 1.    Data set - Seleção

# 2. Data set - Explicação

Esse data set possui 25 colunas, 24 sendo potenciais features e 1 target (satisfação). Ela é uma variável categórica não ordenada e assume 2 valores: satisfeito ou neutro/insatisfeito. Ainda, a target está bem balanceada, estando divida em 43/57 %

Em relação as features, elas variam entre qualitativas e quantitativas, sendo majoritariamente qualitativas e com poucas linhas com valores faltando.

### Colunas

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from mlp import mlp
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from utils import plot_distributions

# carregar os dois conjuntos
df_train = pd.read_csv("train.csv")
df_test  = pd.read_csv("test.csv")

# marcar a origem para poder separar depois
df_train["_source"] = "train"
df_test["_source"]  = "test"

# concatenar
df = pd.concat([df_train, df_test], ignore_index=True)

print("Shape combinado:", df.shape)
print(df["_source"].value_counts())

target = df['satisfaction']

df.columns

### Target bem equilibrada

In [ ]:
plt.figure(figsize=(6,4))
ax = sns.histplot(x=target, stat="percent", discrete=True)

plt.title("Distribuição da variável 'satisfaction' (%)")
plt.xlabel("Satisfação")
plt.ylabel("Porcentagem")
plt.show()

### Preenchendo valores vazios

In [ ]:
print(df.columns[df.isnull().any()]) #observa-se colunas com valores faltando!

### Arrival delay é numérica, utilizaremos a moda para preencher o valor, já que se arrival delay está como null, provavelmente foi 0 e esqueceram de colocar

In [ ]:
df['Arrival Delay in Minutes'] = df['Arrival Delay in Minutes'].fillna(df['Arrival Delay in Minutes'].mode()[0])
print(df.columns[df.isnull().any()]) #roda mais uma vez para garantir

### Visualização features

In [ ]:
# definir colunas
quantitative_cols = ["Age", "Flight Distance", "Departure Delay in Minutes", "Arrival Delay in Minutes"]

ordinal_cols = [
    "Inflight wifi service",
    "Departure/Arrival time convenient",
    "Ease of Online booking",
    "Gate location",
    "Food and drink",
    "Online boarding",
    "Seat comfort",
    "Inflight entertainment",
    "On-board service",
    "Leg room service",
    "Baggage handling",
    "Checkin service",
    "Inflight service",
    "Cleanliness",
]

nominal_cols = ["Gender", "Customer Type", "Type of Travel", "Class"]
target_col = "satisfaction"

# -------- PLOT ORIGINAL --------
plot_distributions(df, quantitative_cols, ordinal_cols, nominal_cols, title_suffix="(Original)", force_ordinal_continuous=False)


Pelas distribuições apresentadas, as variáveis categóricas estão relativamente balanceadas em alguns aspectos, mas apresentam diferenças relevantes em outros. O gênero está equilibrado entre homens e mulheres. Já o tipo de cliente é bastante desbalanceado, com predominância de clientes leais. No tipo de viagem, há mais viagens de negócios do que pessoais. Em relação à classe, as categorias Business e Eco têm proporções próximas, enquanto Eco Plus aparece em bem menor quantidade. A variável de satisfação também é relativamente balanceada, com uma leve maioria de clientes insatisfeitos ou neutros.

Nas variáveis numéricas, observa-se diversidade: a idade segue uma distribuição concentrada entre 20 e 50 anos; a distância do voo é enviesada para valores menores; e os atrasos de partida e chegada apresentam forte concentração em atrasos curtos, com alguns outliers de longos atrasos. Já os serviços avaliativos (como wifi, comida, embarque, conforto de assento, limpeza, etc.) mostram distribuições variadas, mas tendem a concentrar respostas em notas intermediárias a altas, o que sugere certo viés positivo nas avaliações.

# 3. Limpeza de dados e normalização

Claro! Aqui vai uma explicação simples e direta sobre **por que normalizamos cada grupo de variáveis do jeito que fizemos**:

---

### 🔹 Z-Score (média 0, desvio 1)

Usamos em **Age** e nas variáveis **ordinais (avaliações de 1 a 5)**.

* **Por quê?**

  * O Z-score centraliza os dados na média e escala pela variabilidade.
  * Isso coloca todas essas variáveis em uma escala comparável (valores entre -2 e 2, geralmente).
  * É útil quando os dados são aproximadamente simétricos ou queremos destacar desvios em relação à média.

Exemplo: Idades diferentes são comparadas em termos de "quantos desvios padrão acima ou abaixo da média" estão.

---

### 🔹 Min-Max [-1, 1]

Usamos em **Flight Distance** e nos **delays (após log)**.

* **Por quê?**

  * O Min-Max traz os valores para um intervalo fixo, aqui entre -1 e 1.
  * Isso garante que nenhuma variável tenha escala muito maior que as outras.
  * É útil quando a distribuição não é centrada na média, mas queremos que o modelo “veja” tudo na mesma faixa.

---

### 🔹 Log + Min-Max

Usamos em **Departure Delay** e **Arrival Delay**.

* **Por quê?**

  * Atrasos têm distribuição muito enviesada: muitos voos com atraso 0 ou baixo, e poucos voos com atrasos enormes.
  * O log “comprime” esses valores grandes, reduzindo o impacto dos extremos.
  * Depois, aplicamos Min-Max para trazer o resultado para a faixa [-1,1], alinhando com as outras features.

---

### 🔹 One-Hot Encoding (nominais)

Nas variáveis como **Gender, Customer Type, Type of Travel, Class**.

* **Por quê?**

  * São categorias sem ordem (ex.: “Male” ≠ maior que “Female”).
  * O One-Hot cria colunas binárias (`0` ou `1`) para cada categoria, sem necessidade de normalização extra.

---

In [ ]:
# -------- ONE-HOT + TARGET --------
df_encoded = pd.get_dummies(df, columns=nominal_cols, dtype=int)
df_encoded[target_col] = df_encoded[target_col].map({
    "satisfied": 1,
    "neutral or dissatisfied": 0
})

# -------- NORMALIZAÇÃO --------
df_norm = df_encoded.copy()

# Age + ordinais -> Z-score
scaler_z = StandardScaler()
df_norm[["Age"] + ordinal_cols] = scaler_z.fit_transform(df_norm[["Age"] + ordinal_cols])

# Flight Distance -> MinMax [-1,1]
scaler_fd = MinMaxScaler(feature_range=(-1,1))
df_norm["Flight Distance"] = scaler_fd.fit_transform(df_norm[["Flight Distance"]])

# Delays -> log + MinMax [-1,1]
for col in ["Departure Delay in Minutes", "Arrival Delay in Minutes"]:
    df_norm[col] = np.log1p(df_norm[col].clip(lower=0))
    scaler_delay = MinMaxScaler(feature_range=(-1,1))
    df_norm[col] = scaler_delay.fit_transform(df_norm[[col]])

df_norm.head()
# -------- PLOT NORMALIZADO --------
plot_distributions(df_norm, quantitative_cols, ordinal_cols, nominal_cols, title_suffix="(Normalizado)", force_ordinal_continuous=True)

# VG -> DECIDIR SE VAI TER PCA OU NAO (PROVAVELMENTE NAO)

# 4. Implementação MLP

In [ ]:
df_norm.head()

In [ ]:
drop_cols = ["Unnamed: 0", "id", "satisfaction"]

df_train = df_norm[df_norm["_source"] == "train"].drop(columns=["_source"], errors="ignore")
df_test  = df_norm[df_norm["_source"] == "test"].drop(columns=["_source"], errors="ignore")

X_train = df_train.drop(columns=drop_cols, errors="ignore").apply(pd.to_numeric, errors="coerce").fillna(0).values.astype(float)
y_train = df_train["satisfaction"].astype(int).values

X_test = df_test.drop(columns=drop_cols, errors="ignore").apply(pd.to_numeric, errors="coerce").fillna(0).values.astype(float)
y_test = df_test["satisfaction"].astype(int).values if "satisfaction" in df_test.columns else None

model = mlp(
    n_features=X_train.shape[1],
    n_hidden_layers=2,
    n_neurons_per_layer=[32, 16, 1],
    activation="relu",
    loss="mse",
    optimizer="gd",
    epochs=100,
    eta=0.01
)

model.train(X_train, y_train)


# 5. Treinando Modelo

In [ ]:
preds_test = model.test(X_test)

model.evaluate(X_test, y_test, plot_confusion=True, plot_roc=True, preds=preds_test)

# VG -> EXPLICAR O RESTO, TOPICO 6 PRA FREMTE, POSSIVELMENTE MUDAR DE LUGAR OS CODIGOS, ARRUMAR A CASA

# 6. Estratégia de treino e teste

# 7. Curva de erro e visualização

# 8. Avaliação do modelo